# Lab Assignment Two: Exploring Image Data
by:
* Hayden Donofrio
* Riley Bates
* Chandler Choate

## Section 1: Business Understanding


## Section 2: Data Preparation

## Section 3: Data Reduction


## Section 4: Additional Analysis